In [1]:
######################################     Modules     #######################################
import sys
from time import process_time
import os

import verbose_module
sys.path.append('../../.')
import pIRPgym

########################     Instance generator and Environment     #########################
# Instance Generator
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

env_config = {  'M':13,'K':15,'T':12,'F':13,'Q':2000,
                'S':6,'LA_horizon':4,
                'd_max':2000,'hist_window':60,
                'back_o_cost':10000
            }

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# dCVRP Instances

In [2]:
########################     Instance generator and Environment     #########################
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

instances = dict()
instances['Li'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()

### Environment 
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

In [3]:
################################## Policy Evaluation ##################################
''' Parameters '''
verbose = True
show_gap = True

if verbose: verbose_module.routing_instances.print_head(show_gap)

for ss,inst_list in instances.items():
    for instance in inst_list:
        # Upload dCVRP instance
        purchase,benchmark = inst_gen.upload_CVRP_instance(ss, instance)

        if verbose: string = verbose_module.routing_instances.print_inst(ss,instance,benchmark[0],benchmark[1])

        nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
            
        RCL_obj,RCL_veh,RCL_time = pIRPgym.Routing.evaluate_stochastic_policy(pIRPgym.Routing.RCL_Heuristic,
                                                                                           purchase,inst_gen,env,n=30,averages=True,dynamic_p=False)
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    RCL_obj,RCL_veh,RCL_time,show_gap,benchmark,end=True)
        # GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
        # if verbose: string = verbose_module.routing_instances.print_routing_update(string,
        #                                                             GA_obj,len(GA_routes),GA_time,show_gap,benchmark)
        # HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
        # if verbose: string = verbose_module.routing_instances.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes),HyGeSe_time,show_gap,benchmark,True)                                    # Column Generation algorithm
    print('\n')

*************************************  Routing Strategies on Classic Instances  *************************************
--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------


# CVRP Instances

In [2]:
########################     Instance generator and Environment     #########################
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

instances = dict()
instances = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
# instances = [i for i in os.listdir('C:/Users/jm.betancourt/Documents/Research/pIRPgym/pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances.sort()
instances = instances[1:] + [instances[0]]

### Environment 
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

In [3]:
################################## Policy Evaluation ##################################
''' Parameters '''
verbose = True
show_gap = True

ranges = [i for i in range(1,11)]
performance = {'nn':{i:{'FO':list(),'Time':list()} for i in ranges},'RCL':{i:{'FO':list(),'Time':list()} for i in ranges},'GA':{i:{'FO':list(),'Time':list()} for i in ranges}}


if verbose: verbose_module.routing_instances.print_head(show_gap)

for instance in instances:
    # Upload dCVRP instance
    purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

    if verbose: string = verbose_module.routing_instances.print_inst('Uchoa',instance,benchmark[0],benchmark[1])

    nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
    performance['nn'][len(purchase)//100]['FO'].append(nn_obj);performance['nn'][len(purchase)//100]['Time'].append(nn_time)


    RCL_obj,RCL_veh,RCL_time = pIRPgym.Routing.evaluate_stochastic_policy(pIRPgym.Routing.RCL_Heuristic,
                                                                                        purchase,inst_gen,env,n=30,averages=True,dynamic_p=False)
    if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                RCL_obj,RCL_veh,RCL_time,show_gap,benchmark)
    performance['RCL'][len(purchase)//100]['FO'].append(RCL_obj);performance['RCL'][len(purchase)//100]['Time'].append(RCL_time)

    GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                GA_obj,len(GA_routes),GA_time,show_gap,benchmark,end=True)
    performance['GA'][len(purchase)//100]['FO'].append(GA_obj);performance['GA'][len(purchase)//100]['Time'].append(GA_time)
    # HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
    # if verbose: string = verbose_module.routing_instances.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes),HyGeSe_time,show_gap,benchmark,True)                                    # Column Generation algorithm
    

*************************************  Routing Strategies on Classic Instances  *************************************
--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------
n101-k25| 25 	27591 	| 0.00	  26	 52.02	| 0.01	  26.73	 62.52	| 11.70	  26	 29.34	|
n106-k14| 14 	26362 	| 0.00	  14	 9.61	| 0.01	  14.0	 40.48	| 13.10	  14	 7.3	|
n110-k13| 13 	14971 	| 0.00	  13	 28.86	| 0.01	  13.0	 116.25	| 14.10	  13	 18.9	|
n115-k10| 10 	12747 	| 0.00	  10	 51.64	| 0.01	  10.0	 170.78	| 10.50	  10	 33.27	|
n120-k6| 6 	13332 	| 0.00	  6	 19.18	| 0.01	  6.0	 150.59	| 18.20	  6	 13.14	|
n125-k30| 30 	55539 	| 0.00	  32	 23.07	| 0.01	  31.63	 33.94	| 13.40	  31	 16.71	|
n129-k18| 18 	28940 	| 0.00	  18	 24.24	| 0.01	  18.0	 73.48	| 18.50	  18	 17.13	|
n134-k13| 13 	10916 	| 0

Traceback (most recent call last):
  File "/Users/juanbeta/micromamba/envs/SCAEnv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/m6/89fwm0dn5mjbdz4tkj6586bm0000gn/T/ipykernel_8549/1852153929.py", line 30, in <module>
    GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
  File "/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/Experimenting/Juan/../.././pIRPgym/Blocks/Policies/Routing.py", line 184, in HybridGenticAlgorithm
    Population_iter,training_time)
  File "/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/Experimenting/Juan/../.././pIRPgym/Blocks/Policies/Routing.py", line 296, in generate_population
  File "/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/Experimenting/Juan/../.././pIRPgym/Blocks/Policies/

In [4]:
##NN
sum(performance['nn'][1]['FO'])/len()

{'FO': [41944,
  28896,
  19292,
  19330,
  15889,
  68354,
  35956,
  16526,
  16845,
  22533,
  56663,
  30160,
  18704,
  17889,
  25381,
  64976,
  66368,
  27718,
  29253,
  19985,
  65018,
  69192],
 'Time': [0.002272000000000496,
  0.0017440000000021882,
  0.0015300000000024738,
  0.0015909999999976776,
  0.0017160000000018272,
  0.002065999999999235,
  0.0022570000000001755,
  0.0026849999999853935,
  0.0023300000000006094,
  0.0024299999999755073,
  0.00301600000000235,
  0.0034789999999986776,
  0.0033679999999947086,
  0.003270000000043183,
  0.004064999999968677,
  0.005288000000007287,
  0.005199000000004617,
  0.0064699999999788815,
  0.0072999999999865395,
  0.006930000000011205,
  0.00897799999995641,
  0.009531000000038148]}